# version2_1のバグ探索用
方針:testを2017,2018として検証する。

In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.decomposition import PCA  #次元削減用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR
import time
import datetime
import os #ディレクトリ作成用
import xgboost as xgb
import sys





from sklearn.preprocessing import StandardScaler#モデルの評価用に標準化する関数
import scipy.stats#モデルの評価用に標準化する関数

学習、精度検証用データ作成関数

In [4]:
def data_making_clustar(df):#クラスタリングあり、モータ番号、艇番号なし
    result_df=df
    result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
    result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22})#新人のave_st_timeを0.22に
    result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02})#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
    result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02})
    #ダミー変数化
    result_df_dummie=result_df
    race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
    for column, val in race_dummie_df.iteritems():
        result_df_dummie['race_{}'.format(int(column))]=val
    result_df_dummie=result_df_dummie.drop('number_race',axis=1)

    cols=list(result_df_dummie.columns)
    male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す

    #===========================新規、性別の取り出し機能が良くなかったため作り直す
    empty_arr=[0]*len(result_df_dummie)
    for col in male_cols:
        for number in np.arange(0,2,1):
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        male_dummie_df=pd.get_dummies(result_df_dummie[col])#性別をダミー化
        for column, val in male_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

    cols=list(result_df_dummie.columns)



    moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
    boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す

    #boat、moterの情報は使わない、
    numbers=np.arange(1, 100, 1)
    empty_arr=[0]*len(result_df_dummie)
    for col in moter_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)
    for col in boat_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

    #クラスタリング
    #分けてみるクラスタの数は[3,5,7,9]の4個
    #cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
    #学習データのdateを年に変換
    result_df_dummie['date']=pd.to_datetime(result_df_dummie['date'])#日付が文字列なのでdateを日付型に変換
    result_df_dummie['year']=result_df_dummie['date'].dt.year
    result_df_dummie=result_df_dummie[result_df_dummie['year']!=2020]#2020のデータを完全に切り離す。
    #クラスタリングに邪魔だから消したいけど、後々使うものはいったんよけておく
    result=result_df_dummie['result_com'].values#
    money=result_df_dummie['money'].values#
    years=result_df_dummie['year'].values#

    #安全なところに移したら削除する
    result_df_dummie=result_df_dummie.drop('result_com',axis=1)
    result_df_dummie=result_df_dummie.drop('money',axis=1)
    result_df_dummie=result_df_dummie.drop('date',axis=1)
    #クラアスタリング用の学習、予測用のデータの切り分け
    clustar_test_df = result_df_dummie[(result_df_dummie['year']==2018) | ((result_df_dummie['year']==2019) )].copy()#2019,2020のデータを検証用データに。
    clustar_train_df =  result_df_dummie[(result_df_dummie['year']!=2018) & ((result_df_dummie['year']!=2019) )].copy()#そのほかを学習データに

    #年の情報だけ切り分けに使ったからここで消す。
    clustar_test_df=clustar_test_df.drop('year',axis=1)
    clustar_train_df=clustar_train_df.drop('year',axis=1)

    target_num_cluster=[3,5,7,9]#分けるクラスタ数によってモデルの名前を変える
    for num_cluster in target_num_cluster:
        Km = KMeans(random_state=7,n_clusters=num_cluster).fit(clustar_train_df)#rondom_stateはラッキーセブン
        train_pred = Km.predict(clustar_train_df)#rondom_stateはラッキーセブン
        test_pred =Km.predict(clustar_test_df)#rondom_stateはラッキーセブン
        #Km=========================実査に使うときはこれのモデルを会場ごとに保存して使用。

        clustar_train_df['num={}'.format(num_cluster)]=train_pred
        clustar_test_df['num={}'.format(num_cluster)]=test_pred

    #結合して元の形に戻す。
    clustar_df=pd.concat([clustar_train_df, clustar_test_df])
    clustar_df['year']=years
    clustar_df['money']=money
    clustar_df['result_com']=result
    model_df=clustar_df
    return model_df

パラメータ探索関数

In [5]:
#モデルのパラメータ探索関数(XGboost)
def model_score_XGboost_th(version,place_name,result_df):#XGboostの出力を確率のやつを使用したバージョン、閾値の探索も行う。
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','threshold','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])#スコアを格納するdf

    #学習データの切り分け
    test_df = result_df[(result_df['year']==2018) | ((result_df['year']==2019) )]#2019,2020のデータを検証用データに。
    train_df =  result_df[(result_df['year']!=2018) & ((result_df['year']!=2019) )]#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_df=test_df.drop(['year'],axis=1)
    train_df=train_df.drop(['year'],axis=1)

    train_money=pd.Series(train_df['money'])
    test_money=pd.Series(test_df['money'])

    #x,yへの切り分け
    #出現数の分布
    result_com_s=test_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    gain_mean=test_df.groupby('result_com')['money'].mean()
    gain_mean=gain_mean.sort_index()

    gain_median=test_df.groupby('result_com')['money'].median()
    gain_median=gain_median.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                                'result_com_num':result_com_s.values,
                                'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                                'gain_mean':gain_mean.values,
                                'gain_median':gain_median.values,})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため




    for result_com_number in tqdm(result_com_df['result_com'].values):
        #print(result_com_number)
        result_com=result_com_number
        #result_comごとの閾値の決定========================================================================
        #print(result_com_number)
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================

        gain_th=10#利益率の閾値
        result_s=result_com_df[result_com_df['result_com']==result_com]
        buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
        num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_arr=[0]*len(result_train_df)
        i=0
        for result in result_train_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1
        result_train_df['result_com']=result_arr
        result_test_df=test_df.copy()
        result_arr=[0]*len(result_test_df)
        i=0
        for result in result_test_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1

        result_test_df['result_com']=result_arr

        result_train_df['money']=train_money
        result_test_df['money']=test_money
        #学習データラベル変換終わり============================================

        for_arr=np.arange(1,85)
        #for_arr=np.arange(1,100,1)
        accuracy_arr=[0]*len(for_arr)
        target_per_arr=[0]*len(for_arr)
        pred_0=[0]*len(for_arr)
        gain_arr=[0]*len(for_arr)
        model_gain_arr=[0]*len(result_test_df)
        test_gain_arr=test_money.values
        #depths_arr=[4,5,6,7,8]
        #depths_arr=[5,6,8]
        depths_arr=[5,8]
        for depth in depths_arr:
            for sum_target_per in for_arr:

                index=sum_target_per-1
                #target_per=50+sum_target_per
                target_per=100+(sum_target_per*2)
                target_per_arr[index]=target_per

                #モデルの評価指標値を格納するseries======================
                model_score_s=pd.Series(index=['target_com','depth','target_per','threshold','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'], dtype='float64')
                model_score_s['target_com']=result_com#目標としているresult_comラベル番号
                model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
                model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
                #======================
                #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
                # 一層目の判別機のtrainデータ　:terget_result_df
                target_df=result_train_df#ベースのデータフレームをコピー
                target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
                target_1_df=target_df[target_df['result_com']==1]
                len_1=len(target_1_df)
                target_0_df=target_df[target_df['result_com']==0]
                len_0=len(target_0_df)
                target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
                target_train_df=pd.concat([target_1_df, target_0_df])
                #学習＆予測ぱーと========================================================================
                #==========================================================================================================================================
                #データの切り分け
                target_x_train=target_train_df.drop('money',axis=1)
                target_x_train=target_x_train.drop('result_com',axis=1)
                target_x_test=result_test_df.drop('money',axis=1)
                target_x_test=target_x_test.drop('result_com',axis=1)

                target_y_train=target_train_df['result_com']
                target_y_test=result_test_df['result_com']
                train_x, valid_x, train_y, valid_y = train_test_split(target_x_train, target_y_train, test_size=0.2, shuffle=True, random_state=7)#学習データ内でさらに分割してロスをもとに修正をする。

                #XGboostのデータ型に変換する
                train = xgb.DMatrix(train_x, label=train_y)#学習用
                valid = xgb.DMatrix(valid_x, label=valid_y)#学習時のロス修正用
                test = xgb.DMatrix(target_x_test, label=target_y_test)#実際に使った時の利益率の算出用

                #xgb.config_context(verbosity=0)
                param = {'max_depth': depth, #パラメータの設定
                                 #'eta': 1.8,
                                 #'eta': 0.8,
                                 'eta': 1.3,
                                 #'eta': 0.2,
                                 #'objective': 'binary:hinge',
                                 'objective': 'binary:logistic',#確率で出力
                                 'eval_metric': 'logloss',
                                 'verbosity':0,
                                 'subsample':0.8,
                                 'nthread':10,
                                 'gpu_id':0,
                                 'seed':7,
                                 'tree_method':'gpu_hist'
                                }
                evallist = [(valid, 'eval'), (train, 'train')]#学習時にバリデーションを監視するデータの指定。
                #bst = xgb.train(param, train,num_boost_round=1000,early_stopping_rounds=30)
                #num_round = 10000
                num_round = 400
                bst = xgb.train(param, train,num_round,evallist, early_stopping_rounds=30, verbose_eval=0 )
                #bst = xgb.train(param, train,num_round,evallist, verbose=100,early_stopping_rounds=30 )
                #RF = RandomForestClassifier(random_state=1,n_estimators=1000,max_depth=depth)
                #RF = RF.fit(target_x_train,target_y_train)


                # 未知データに対する予測値
                #predict_y_test = RF.predict(target_x_test)
                predict_y_test=bst.predict(test)

                #==========================================================================================================================================
                #==========================================================================================================================================
                #==========================================================================================================================================
                #==========================================================================================================================================

                #[1]の正答率を見る
                pred_test_df=pd.DataFrame({'pred_proba':predict_y_test#確率分布での出力
                                          , 'test':target_y_test})

                #th_arr=[0.1,0.3,0.5,0.6,0.7,0.8,0.9]
                #th_arr=[0.01,0.03,0.05,0.07,0.9,0.1,0.13]#探索結果待ち、、、、、
                th_arr=[0.85,0.9,0.92]
                for th in th_arr:
                    trans_df=pred_th_trans(pred_test_df,th)
                    num_1=len(trans_df[trans_df['test']==1])
                    count=0
                    #追加　配当金の情報も考慮する。
                    gain_index=0
                    model_gain_arr=[0]*len(result_test_df)
                    for _, s in trans_df.iterrows():
                        if ((s['pred']==1) and (s['test']==1)):#もし購買しているかつ的中をしていたら・・・
                            count+=1#的中回数
                            model_gain_arr[gain_index]=test_gain_arr[gain_index]
                        gain_index+=1
                    #print('test accyracy: {}'.format((count/num_1)*100))
                    gain_arr[index]=sum(model_gain_arr)
                    accuracy_arr[index]=(count/num_1)*100
                    try:
                        pred_0[index]=trans_df['pred'].value_counts()[0]
                    except:
                        pred_0[index]=0
                    #scoreのseriesに情報書き込み==================
                    model_score_s['threshold']=th
                    model_score_s['総収益']=sum(model_gain_arr)
                    #model_score_s['投資金額']=100*sum(predict_y_test)
                    model_score_s['投資金額']=100*trans_df['pred'].sum()
                    model_score_s['出現数']=sum(target_y_test)
                    #model_score_s['購買予測数']=sum(predict_y_test)
                    model_score_s['購買予測数']=trans_df['pred'].sum()
                    model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                    model_score_s['購買的中率']=(count/trans_df['pred'].sum())*100
                    model_score_s['的中数']=count
                    model_score_df=model_score_df.append(model_score_s,ignore_index=True)
    #モデルの「スコアを保存
    #model_score_df.to_csv('{}_model_score.csv'.format(place), encoding='utf_8_sig')
    dir_path = "check_csv/{place_name}/{place_name}_model_score_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先#使用するデータの読み込み
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

## その他

In [6]:
#バージョンとは関係のない、データの加工関数内で使う関数====================================================================================================================================================================================================
def make_PCA_df(PCA_arr):#PCAで削減したものは二次元配列で帰ってくるので、それをデータフレームにして返す関数
    X=[0]*len(PCA_arr)
    Y=[0]*len(PCA_arr)
    index=0
    for arr in PCA_arr:
        X[index]=arr[0]
        Y[index]=arr[1]
        index+=1
    return pd.DataFrame({'X':X,'Y':Y})

#dateのカラムを年だけに変換するやつ
def trans_date_type(df):
    df['date']=pd.to_datetime(df['date'])#日付が文字列なのでdateを日付型に変換
    df['year']=df['date'].dt.year
    df=df.drop('date',axis=1)
    return df

#いったん仮でいちいちほかの機能を探さないでいいようにおいておくよ（ハート）
#dateのカラムからの情報の抽出
#def trans_date_type(df):
#    df['date']=pd.to_datetime(df['date'])#日付が文字列なのでdateを日付型に変換
#    df['year']=df['date'].dt.year
#    df['month']=df['date'].dt.month
#    df['day']=df['date'].dt.day
#    df=df.drop('date',axis=1)
#    return df

#閾値を渡して、その値以上を1、未満を0に置き変える。
def pred_th_trans(pred_df,th):
    #引数として予測結果のdeと、変換したい閾値を渡す。
    trans_df=pred_df.copy()
    trans_df.loc[trans_df['pred_proba'] >= th, 'pred'] = 1
    trans_df.loc[~(trans_df['pred_proba']  >=  th), 'pred'] = 0
    return trans_df

検証用データ格納用ディレクトリ作成

In [7]:
def get_place_master():
    place_dict={"01":"kiryu"
                ,"02":"toda"
                ,"03":"edogawa"
                ,"04":"heiwazima"
                ,"05":"tamagawa"
                ,"06":"hamanako"
                ,"07":"gamagori"
                ,"08":"tokoname"
                ,"09":"tu"
                ,"10":"mikuni"
                ,"11":"biwako"
                ,"12":"suminoe"
                ,"13":"amagasaki"
                ,"14":"naruto"
                ,"15":"marugame"
                ,"16":"kozima"
                ,"17":"miyazima"
                ,"18":"tokuyama"
                ,"19":"simonoseki"
                ,"20":"wakamatu"
                ,"21":"asiya"
                ,"22":"fukuoka"
                ,"23":"karatu"
                ,"24":"omura"}
    return place_dict

def get_place_num_master():
    place_num_dict={1:"kiryu"
                ,2:"toda"
                ,3:"edogawa"
                ,4:"heiwazima"
                ,5:"tamagawa"
                ,6:"hamanako"
                ,7:"gamagori"
                ,8:"tokoname"
                ,9:"tu"
                ,10:"mikuni"
                ,11:"biwako"
                ,12:"suminoe"
                ,13:"amagasaki"
                ,14:"naruto"
                ,15:"marugame"
                ,16:"kozima"
                ,17:"miyazima"
                ,18:"tokuyama"
                ,19:"simonoseki"
                ,20:"wakamatu"
                ,21:"asiya"
                ,22:"fukuoka"
                ,23:"karatu"
                ,24:"omura"}
    return place_num_dict


In [17]:
place_master=get_place_master()
for place in place_master.items():
    #print(place[0],place[1],'\n')
    place_name=place[1]
    dir_path = "check_csv/{place_name}/".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
    if os.path.exists(dir_path)==False:
        os.makedirs(dir_path)
    else:
        pass
    dir_path = "check_pickle/{place_name}/".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
    if os.path.exists(dir_path)==False:
        os.makedirs(dir_path)
    else:
        pass
    

In [19]:
result_filepath="../../..//bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)
result_df=data_making_clustar(result_base_df)

#model_score_XGboost_th(version,place_name,result_df)#閾値を決めて変換するver


2019

In [34]:
def version_2_1_2(version,place_name,base_df):#閾値で予測を変えるバージョン
    result_df=data_making_clustar(base_df)
    print(result_df['year'].max())
    model_score_XGboost_th(version,place_name,result_df)#閾値を決めて変換するver

    
#データ格納用のディレクトリ作り
version='V2_1_2'#バージョン


# # place_master=master.get_place_master()
# # for place in place_master.items():
# #place_name=place[1]
# place_name='tokuyama'
# ################################################========================================================================================================================
# ################################################========================================================================================================================
# ################################################========================================================================================================================
# ################################################========================================================================================================================
# #各会場の学習データの読み込みと加工
# result_filepath="../../..//bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
# result_base_df=pd.read_csv(result_filepath)
# result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)
# result_base_df=result_base_df[result_base_df['year']!=2020]#2020のデータを完全に切り離す。
# version_2_1_2(version,place_name,result_base_df):


#データ格納用のディレクトリ作り
version='V2_1_2'#バージョン
place_master=get_place_master()
for place in place_master.items():
    
    place_name=place[1]
    print("place_name",place_name)
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    #各会場の学習データの読み込みと加工
    result_filepath="../../..//bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
    result_base_df=pd.read_csv(result_filepath)
    result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)
    version_2_1_2(version,place_name,result_base_df)

place_name kiryu


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

2019
kiryu


100%|███████████████████████████████████████████████████████████████████████████████| 28/28 [5:19:23<00:00, 684.42s/it]


place_name toda


...

100%|███████████████████████████████████████████████████████████████████████████████| 28/28 [1:33:59<00:00, 201.42s/it]


place_name omura


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

2019
omura


100%|███████████████████████████████████████████████████████████████████████████████| 28/28 [1:36:00<00:00, 205.72s/it]


元データ読み込み

### そのまま同ノートブックでモデルのpickle保存と予測を行う。

In [13]:
#モデルスコア分析用の関数
def model_analysis(score_df,place_name,version):
    #基準をクリアしたモデルを格納するdf
    depths_arr=[5,8]
    th_arr=[0.85,0.9,0.92]
    #depths_arr=[4,5,6,7,8]
    result_com_arr=np.arange(1, 29)

    #基本敵に今回からは出現数とかはresult_comのdfから作成しないでスコアのdfからしゅとくする。

    model_para_df=pd.DataFrame(columns=['target_com','depth','target_per','threshold','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
    #model_para_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
    for target_com in result_com_arr:
        target_com_df=score_df[score_df['target_com']==target_com]
        num_com= target_com_df['出現数'].values[0]  #出現数
        #閾値の作成===============================================================
        #gain_th=110#利益率の閾値(メインモデル)
        gain_th=115#利益率の閾値(メインモデル)
        #result_s=result_com_df[result_com_df['result_com']==target_com]
        buy_accuracy_th=5#result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値(いったん決めで5%とする)
        num_tp_th=num_com*0.15#あたった回数の閾値(出現回数の15%が的中)

        diff_gain_th=105#利益率の閾値(前後モデル)
        diff_buy_accuracy_th=5#result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値(いったん決めで5%とする)
        diff_num_tp_th=num_com*0.15#あたった回数の閾値(出現回数の15%が的中)

        #==========================================================
        for target_depth in depths_arr:
            target_com_depth_df=target_com_df[target_com_df['depth']==target_depth]

            for th in th_arr:
                target_com_depth_th_df=target_com_depth_df[target_com_depth_df['threshold']==th]

                len_df=len(target_com_depth_th_df)
                df=target_com_depth_th_df.copy()
                df['number_i']=np.arange(0,len_df,1)
                #display(df)
                for _, row in df.iterrows():
                    #if ((row['number_i']==0) or (row['number_i']==1) or (row['number_i']==len_df-2) or (row['number_i']==len_df-1)):
                    if ((row['number_i']==0) or (row['number_i']==len_df-1)):#前後モデルが存在しないのは評価に含めない（）
                        pass
                    else:
                        #前後のモデルのスコア(今回から±１にした)===================================
                        #diff_m2_row=df[df['number_i']==(row['number_i']-2)]
                        diff_m1_row=df[df['number_i']==(row['number_i']-1)]
                        diff_p1_row=df[df['number_i']==(row['number_i']+1)]
                        #diff_p2_row=df[df['number_i']==(row['number_i']+2)]
                        #diff_models=[diff_m2_row,diff_m1_row,diff_p1_row,diff_p2_row]
                        diff_models=[diff_m1_row,diff_p1_row]
                        #===================================================
                        if ((row['利益率']>gain_th) and (row['購買的中率']>buy_accuracy_th) and (row['的中数']>=num_tp_th)):#初めに真ん中のモデルの性能評価
                            #前後モデルの評価
                            flag_arr=[0,0]
                            for index in range(len(flag_arr)):

                                diff_model=diff_models[index].iloc[0,:]
                                if ((diff_model['利益率']>diff_gain_th) and (diff_model['購買的中率']>diff_buy_accuracy_th) and (diff_model['的中数']>=diff_num_tp_th)):#前後モデルの性能評価
                                    flag_arr[index]=1
                                else:
                                    pass
                            #if sum(flag_arr)==2:
                            if sum(flag_arr)>=1:
                                model_para_df=model_para_df.append(row)

                        else:
                            pass
    dir_path = "check_csv/{place_name}/good_model_{place_name}_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先#使用するデータの読み込み
    model_para_df.to_csv(dir_path, encoding='utf_8_sig')

def use_model_para(good_model_df,place_name,version):#実査に使用するのにどのモデルが最適か判別するパラメータを作成する関数(製作中)
    use_model_df=pd.DataFrame(columns=['target_com','depth','target_per','threshold','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数','的中数_std','利益率_std','score'])
    print(place_name)
    for com in sorted(good_model_df['target_com'].value_counts().index):
        target_com_df=good_model_df[good_model_df['target_com']==com]
        #sc = StandardScaler()
        #sc =sc.fit(target_com_df['的中数'].values)
        target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
        #target_com_df['的中数_std']=sc.transform(target_com_df['的中数'].values)
        target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)

        #target_com_df['利益率_std']=sc.fit_transform(target_com_df['利益率'].values)
        target_com_df['score']=target_com_df['的中数_std']+target_com_df['利益率_std']
        if len(target_com_df)>=3:
            target_com_df=target_com_df[target_com_df['利益率_std']>0]#なるべく利益率を重視したモデル、偏差の中央以下の基準の利益率のモデルは候補に入れない。
            use_model_df_row=target_com_df[target_com_df['score']==target_com_df['score'].max()]
        elif len(target_com_df)==2:#モデルが二つしかなかった時、最良のモデルが二つできてまう。
            use_model_df_row=target_com_df[target_com_df['利益率']==target_com_df['利益率'].max()]#そんなときは利益率で選ぶ
        elif len(target_com_df)==1:#モデルが1つしかなかった時、そのまま代入
            use_model_df_row=target_com_df
        use_model_df=pd.concat([use_model_df, use_model_df_row])

    dir_path = "check_csv/{place_name}/use_model_{place_name}_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先#使用するデータの読み込み
    use_model_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

### 分析対象のモデルスコアシート

In [12]:

#データ格納用のディレクトリ作り
version='V2_1_2'#バージョン
place_master=get_place_master()
for place in tqdm(place_master.items()):
    place_name=place[1]
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    #各会場の学習したモデルのスコアの読み込み
    score_file_path= "check_csv/{place_name}/{place_name}_model_score_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先#使用するデータの読み込み
    score_df=pd.read_csv(score_file_path)
    score_df=score_df.drop(["Unnamed: 0"],axis=1)
    
    
    model_analysis(score_df,place_name,version)

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [02:09<00:00,  5.38s/it]


In [17]:

## good_modelのスコアからさらに一つのcomごとに一つのモデルに絞り込む
version='V2_1_2'#バージョン


place_master=get_place_master()
for place in tqdm(place_master.items()):
    place_name=place[1]
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    #各会場の成績の良かったモデルのスコアの読み込み
    dir_path = "check_csv/{place_name}/good_model_{place_name}_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先#使用するデータの読み込み
    good_score_df=pd.read_csv(dir_path)
    good_score_df=good_score_df.drop(["Unnamed: 0"],axis=1)
    
    
    use_model_para(good_score_df,place_name,version)

  0%|                                                                                           | 0/24 [00:00<?, ?it/s]<ipython-input-13-7438146856a6>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
<ipython-input-13-7438146856a6>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
<ipython-input-13-7438146856a6>:82: SettingWithCopyWarning: 
A value is trying to b

 21%|█████████████████▎                                                                 | 5/24 [00:00<00:00, 43.42it/s]<ipython-input-13-7438146856a6>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
<ipython-input-13-7438146856a6>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
<ipython-input-13-7438146856a6>:82: SettingWithCopyWarning: 
A value is trying to b

kiryu
toda
edogawa
heiwazima
tamagawa
hamanako
gamagori
tokoname


<ipython-input-13-7438146856a6>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
<ipython-input-13-7438146856a6>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['score']=target_com_df['的中数_std']+target_com_df['利益率_std']
<ipython-input-13-7438146856a6>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

 42%|██████████████████████████████████▏                                               | 10/24 [00:00<00:00, 43.04it/s]<ipython-input-13-7438146856a6>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
<ipython-input-13-7438146856a6>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
<ipython-input-13-7438146856a6>:82: SettingWithCopyWarning: 
A value is trying to b

...

<ipython-input-13-7438146856a6>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
<ipython-input-13-7438146856a6>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
<ipython-input-13-7438146856a6>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

<ipython-input-13-7438146856a6>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
<ipython-input-13-7438146856a6>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
<ipython-input-13-7438146856a6>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

<ipython-input-13-7438146856a6>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['score']=target_com_df['的中数_std']+target_com_df['利益率_std']
<ipython-input-13-7438146856a6>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
<ipython-input-13-7438146856a6>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 43.06it/s]

tokuyama
simonoseki
wakamatu
asiya
fukuoka
karatu
omura


## use_modelをもとにモデルの保存をする。

### モデル保存系の関数

In [18]:
def save_model_V2_1_2(result_base_df,use_model_df,place_name,version):
    #探査結果から学習したモデルを保存する関数、
    print(place_name)
    #==============================================================================
    #学習関数で場所ごとにバージョンに対応した学習データを作る
    result_df=data_making_clustar(result_base_df)
    print(result_df['year'].max())
    #==============================================================================
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','threshold','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])#スコアを格納するdf

    #学習データの切り分け
    test_df = result_df[(result_df['year']==2018) | ((result_df['year']==2019) )]#2019,2020のデータを検証用データに。
    train_df =  result_df[(result_df['year']!=2018) & ((result_df['year']!=2019) )]#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_df=test_df.drop(['year'],axis=1)
    train_df=train_df.drop(['year'],axis=1)

    train_money=pd.Series(train_df['money'])
    test_money=pd.Series(test_df['money'])

    # #x,yへの切り分け
    # #出現数の分布
    # result_com_s=test_df['result_com'].value_counts()
    # result_com_s=result_com_s.sort_index()
    # gain_mean=test_df.groupby('result_com')['money'].mean()
    # gain_mean=gain_mean.sort_index()
    #
    # gain_median=test_df.groupby('result_com')['money'].median()
    # gain_median=gain_median.sort_index()
    # result_com_df=pd.DataFrame({'result_com':result_com_s.index,
    #                             'result_com_num':result_com_s.values,
    #                             'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
    #                             'gain_mean':gain_mean.values,
    #                             'gain_median':gain_median.values,})
    # result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため
    for index, model_row in use_model_df.iterrows():
        #パラメータ＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
        #======================================================================================
        #result_com=int(model_row['target_com'])
        result_com=int(model_row['target_com'])
        depth=int(model_row['depth'])
        target_per=int(model_row['target_per'])
        th=float(model_row['threshold'])

        #======================================================================================
        #======================================================================================
        #======================================================================================


        # gain_th=10#利益率の閾値
        # result_s=result_com_df[result_com_df['result_com']==result_com]
        # buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
        # num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_arr=[0]*len(result_train_df)
        i=0
        for result in result_train_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1
        result_train_df['result_com']=result_arr
        result_test_df=test_df.copy()
        result_arr=[0]*len(result_test_df)
        i=0
        for result in result_test_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1

        result_test_df['result_com']=result_arr

        result_train_df['money']=train_money
        result_test_df['money']=test_money
        #学習データラベル変換終わり============================================

        for_arr=np.arange(1,85)
        #for_arr=np.arange(1,100,1)
        accuracy_arr=[0]*len(for_arr)
        target_per_arr=[0]*len(for_arr)
        pred_0=[0]*len(for_arr)
        gain_arr=[0]*len(for_arr)
        model_gain_arr=[0]*len(result_test_df)
        test_gain_arr=test_money.values


        #モデルの評価指標値を格納するseries======================
        model_score_s=pd.Series(index=['target_com','depth','target_per','threshold','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'], dtype='float64')
        model_score_s['target_com']=result_com#目標としているresult_comラベル番号
        model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
        model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
        #======================
        #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
        # 一層目の判別機のtrainデータ　:terget_result_df
        target_df=result_train_df#ベースのデータフレームをコピー
        target_df=target_df.sample(frac=1,random_state=7)#シャッフル、時系列の偏りを無くす
        target_1_df=target_df[target_df['result_com']==1]
        len_1=len(target_1_df)
        target_0_df=target_df[target_df['result_com']==0]
        len_0=len(target_0_df)
        target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
        target_train_df=pd.concat([target_1_df, target_0_df])
        #学習＆予測ぱーと========================================================================
        #==========================================================================================================================================
        #データの切り分け
        target_x_train=target_train_df.drop('money',axis=1)
        target_x_train=target_x_train.drop('result_com',axis=1)
        target_x_test=result_test_df.drop('money',axis=1)
        target_x_test=target_x_test.drop('result_com',axis=1)

        target_y_train=target_train_df['result_com']
        target_y_test=result_test_df['result_com']
        train_x, valid_x, train_y, valid_y = train_test_split(target_x_train, target_y_train, test_size=0.2,shuffle=True, random_state=7)#学習データ内でさらに分割してロスをもとに修正をする。

        #XGboostのデータ型に変換する
        train = xgb.DMatrix(train_x, label=train_y)#学習用
        valid = xgb.DMatrix(valid_x, label=valid_y)#学習時のロス修正用
        test = xgb.DMatrix(target_x_test, label=target_y_test)#実際に使った時の利益率の算出用

        #xgb.config_context(verbosity=0)
        param = {'max_depth': depth, #パラメータの設定
                         #'eta': 1.8,
                         #'eta': 0.8,
                         'eta': 1.3,
                         #'eta': 0.2,
                         #'objective': 'binary:hinge',
                         'objective': 'binary:logistic',#確率で出力
                         'eval_metric': 'logloss',
                         'verbosity':0,
                         'subsample':0.8,
                         'nthread':10,
                         'gpu_id':0,
                         'seed':7,
                         'tree_method':'gpu_hist'
                        }
        evallist = [(valid, 'eval'), (train, 'train')]#学習時にバリデーションを監視するデータの指定。
        num_round = 400
        bst = xgb.train(param, train,num_round,evallist, early_stopping_rounds=30, verbose_eval=0 )

        # 未知データに対する予測値
        #predict_y_test = RF.predict(target_x_test)
        predict_y_test=bst.predict(test)


        #==========================================================================================================================================
        #==========================================================================================================================================

        #pickle_path="../bot_database/{place_name}/model_pickle_{place_name}/com{com}_{depth}_{target_per}_{th}_{place_name}.pickle".format(place_name=place_name,com=result_com,depth=depth,target_per=target_per,th=th)#モデルを保存
        pickle_path="check_pickle/{place_name}/com{com}_{depth}_{target_per}_{th}_{place_name}.sav".format(place_name=place_name,com=result_com,depth=depth,target_per=target_per,th=th)#モデルを保存
        #print(pickle_path)
        pickle.dump(bst, open(pickle_path, "wb"))#モデルの保存
        #with open(pickle_path, 'wb') as model_file:
        #    pickle.dump(bst, model_file)
        #==========================================================================================================================================
        #==========================================================================================================================================



        #[1]の正答率を見る
        pred_test_df=pd.DataFrame({'pred_proba':predict_y_test#確率分布での出力
                                  , 'test':target_y_test})
        trans_df=pred_th_trans(pred_test_df,th)
        num_1=len(trans_df[trans_df['test']==1])
        count=0
        #追加　配当金の情報も考慮する。
        gain_index=0
        model_gain_arr=[0]*len(result_test_df)
        for _, s in trans_df.iterrows():
            if ((s['pred']==1) and (s['test']==1)):#もし購買しているかつ的中をしていたら・・・
                count+=1#的中回数
                model_gain_arr[gain_index]=test_gain_arr[gain_index]
            gain_index+=1
        #print('test accyracy: {}'.format((count/num_1)*100))
        gain_arr[index]=sum(model_gain_arr)
        accuracy_arr[index]=(count/num_1)*100
        try:
            pred_0[index]=trans_df['pred'].value_counts()[0]
        except:
            pred_0[index]=0
        #scoreのseriesに情報書き込み==================
        model_score_s['threshold']=th
        model_score_s['総収益']=sum(model_gain_arr)
        #model_score_s['投資金額']=100*sum(predict_y_test)
        model_score_s['投資金額']=100*trans_df['pred'].sum()
        model_score_s['出現数']=sum(target_y_test)
        #model_score_s['購買予測数']=sum(predict_y_test)
        model_score_s['購買予測数']=trans_df['pred'].sum()
        model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
        model_score_s['購買的中率']=(count/trans_df['pred'].sum())*100
        model_score_s['的中数']=count
        model_score_df=model_score_df.append(model_score_s,ignore_index=True)
    #モデルの「スコアを保存
    #model_score_df.to_csv('{}_model_score.csv'.format(place), encoding='utf_8_sig')
    dir_path = "check_csv/{place_name}/check_{place_name}_model_score_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先#使用するデータの読み込み
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

In [20]:

#データ格納用のディレクトリ作り
version='V2_1_2'#バージョン


place_master=get_place_master()
for place in tqdm(place_master.items()):
    place_name=place[1]
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    ################################################========================================================================================================================
    
    dir_path = "../../..//bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#学習のためのベースになるリザルトデータ
    result_base_df=pd.read_csv(dir_path)
    result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)
    
    #各会場の成績の良かったモデルのスコアの読み込み
    model_dir_path = "check_csv/{place_name}/use_model_{place_name}_{V}.csv".format(place_name=place_name,V=version)#使用するモデルのパラメータ読み込み
    use_model_df=pd.read_csv(model_dir_path)
    use_model_df=use_model_df.drop(["Unnamed: 0"],axis=1)

    save_model_V2_1_2(result_base_df,use_model_df,place_name,version)
    #model_analysis.use_model_para(good_score_df,place_name,version)

  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

kiryu
2019


  4%|███▍                                                                               | 1/24 [00:07<02:51,  7.44s/it]

toda
2019


  8%|██████▉                                                                            | 2/24 [00:15<02:50,  7.76s/it]

...

 92%|███████████████████████████████████████████████████████████████████████████▏      | 22/24 [03:11<00:19,  9.53s/it]

karatu
2019


 96%|██████████████████████████████████████████████████████████████████████████████▌   | 23/24 [03:19<00:09,  9.00s/it]

omura
2019


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [03:31<00:00,  8.81s/it]
